<a href="https://colab.research.google.com/github/Yuvaraj-IIT-Madras/RAG-Llama-Streamlit/blob/main/llama3_2_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Packages

In [ ]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 21.0 MB/s 

### Import Libraries

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('groq')

# GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [ ]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["./Basics_of_finance.pdf"])
documents = reader.load_data()

https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [ ]:
len(documents)

86

In [ ]:
documents[4].metadata

{'page_label': '5',
 'file_name': 'Basics_of_finance.pdf',
 'file_path': 'Basics_of_finance.pdf',
 'file_type': 'application/pdf',
 'file_size': 1879774,
 'creation_date': '2024-10-11',
 'last_modified_date': '2024-10-11'}

### Chunking

In [ ]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [ ]:
len(nodes)

86

In [ ]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'Basics_of_finance.pdf',
 'file_path': 'Basics_of_finance.pdf',
 'file_type': 'application/pdf',
 'file_size': 1879774,
 'creation_date': '2024-10-11',
 'last_modified_date': '2024-10-11'}

https://chunkviz.up.railway.app/

### Embedding Model

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://huggingface.co/spaces/mteb/leaderboard

### Define LLM Model

In [ ]:
llm = Groq(model="llama-3.2-1b-preview", api_key=GROQ_API_KEY)

https://console.groq.com/docs/models

https://console.groq.com/keys

### Configure Service Context

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

### Create Vector Store Index

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/86 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

#### Persist/Save Index

In [ ]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

#### Define Storage Context

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/

#### Load Index

In [ ]:
index = load_index_from_storage(storage_context, service_context=service_context)

### Define Query Engine

In [ ]:
query_engine = index.as_query_engine(service_context=service_context)

https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/

#### Feed in user query

https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/#viewingcustomizing-prompts

In [ ]:
query = "Explain market bonds?"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

Market bonds are a type of debt instrument that allows investors to lend money to governments, corporations, or other entities to finance their activities. They are essentially loans with a fixed interest rate and a maturity date. Unlike traditional loans, market bonds are traded on the open market, allowing investors to buy and sell them throughout their lifetime.

Market bonds typically have the following characteristics:

1. **Fixed interest rate**: The interest rate paid by the issuer is fixed and guaranteed by the government or entity issuing the bond.
2. **Maturity date**: The bond has a specific maturity date, after which the issuer repays the face value of the bond to the investor.
3. **Trading**: Market bonds are traded on the open market, allowing investors to buy and sell them throughout their lifetime.
4. **Risk**: Market bonds carry a risk of default by the issuer, although this risk is typically managed through the issuance of government bonds or other securities with sim

https://itsjb13.medium.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

https://docs.llamaindex.ai/en/stable/optimizing/production_rag/